In [6]:
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import os

file_exts = ['.jpg', '.png', '.cr2']

images_paths = [
    '/home/jack/Mounts/RemoteS3/kona_coffee_images/leaves',
]

old_df = None
if os.path.isfile('classified.csv'):
    old_df = pd.read_csv('classified.csv')

box_size = 64

def recursive_list(dir_path, any=False):
    file_list = []
    for root, directories, filenames in os.walk(dir_path):
        for filename in filenames:
            file_path = os.path.join(root, filename)
            if os.path.isfile(file_path):
                if any:
                    file_list.append(file_path)
                elif os.path.splitext(file_path)[1].lower() in file_exts:
                    file_list.append(file_path)
    return file_list

images = [image for path in images_paths for image in recursive_list(path)]

print(f'Total Images: {len(images)}')

np.random.shuffle(images)

# remove images that have already been classified
if old_df is not None:
    images = [image for image in images if image not in old_df['path'].unique().tolist()]

print(f'Images to classify: {len(images)}')

Total Images: 316
Images to classify: 299


In [7]:
cv2.namedWindow('image')
cv2.resizeWindow('image', 800, 640)

def get_display_image(img, x, y, size):
    # grab image from x - size*2, y + size*2
    
    r1 = 10
    r2 = 11

    x1 = x - size * r1
    y1 = y - size * r1
    x2 = x + size * r2
    y2 = y + size * r2
    
    cropped = img.crop((x1, y1, x2, y2))
    
    # cropped to cv2
    cropped = np.array(cropped)
    cropped = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
    
    # draw grid lines on image
    for i in range(0, cropped.shape[0], size):
        cv2.line(cropped, (0, i), (cropped.shape[1], i), (0, 255, 0), 1)
    for i in range(0, cropped.shape[1], size):
        cv2.line(cropped, (i, 0), (i, cropped.shape[0]), (0, 255, 0), 1)
        
    # draw red box in center
    cv2.rectangle(cropped, (size*r1, size*r1), (size*r2, size*r2), (0, 0, 255), 1)
    
    #cropped = cv2.resize(cropped, (cropped.shape[1] * 2, cropped.shape[0] * 2))
    return cropped


def draw_rect(img, x, y, size, color):
    img = np.array(img)
    
    # draw filled rectangle
    img[y:y+size, x:x+size] = color

    return Image.fromarray(img)


image_index = 0
x = 0
y = 0
data = []
full_image = Image.open(images[image_index])
show_image = get_display_image(full_image, x, y, box_size)
go_next = False

while True:

    cv2.imshow("image", show_image)
    k = cv2.waitKey(50) & 0xFF
    
    # if escape key, exit
    if k == 27:
        break
    
    # skip the box
    elif k == ord('s'):
        go_next = True
    
    # mark as normal with no rust
    elif k == ord('n'):
        data.append([images[image_index], x, y, box_size, 0])
        full_image = draw_rect(full_image, x, y, box_size, (0, 255, 0))
        go_next = True

    # mark as rust
    elif k == ord('r'):
        data.append([images[image_index], x, y, box_size, 1])
        full_image = draw_rect(full_image, x, y, box_size, (255, 0, 0))
        go_next = True

    if go_next:
        go_next = False
        
        # go to the next box
        x += box_size
        if x > full_image.width - box_size:
            x = 0
            y += box_size
            if y > full_image.height - box_size:
                y = 0
                image_index += 1
                if image_index >= len(images):
                    break
                full_image = Image.open(images[image_index])

        show_image = get_display_image(full_image, x, y, box_size)

# close all open windows
cv2.destroyAllWindows()

In [8]:
df = pd.DataFrame(data, columns=['path', 'x', 'y', 'size', 'rust'])

# add new data to old data
if old_df is not None:
    df = pd.concat([old_df, df], ignore_index=True)


df.to_csv('classified.csv', index=False)

df

,path,x,y,size,rust
0,/home/jack/Mounts/RemoteS3/kona_coffee_images/...,0,0,64,0
1,/home/jack/Mounts/RemoteS3/kona_coffee_images/...,64,0,64,0
2,/home/jack/Mounts/RemoteS3/kona_coffee_images/...,128,0,64,0
3,/home/jack/Mounts/RemoteS3/kona_coffee_images/...,192,0,64,0
4,/home/jack/Mounts/RemoteS3/kona_coffee_images/...,256,0,64,0
...,...,...,...,...,...
47003,/home/jack/Mounts/RemoteS3/kona_coffee_images/...,3712,2944,64,1
47004,/home/jack/Mounts/RemoteS3/kona_coffee_images/...,3776,2944,64,0
47005,/home/jack/Mounts/RemoteS3/kona_coffee_images/...,3840,2944,64,0
47006,/home/jack/Mounts/RemoteS3/kona_coffee_images/...,3904,2944,64,0


In [9]:
df['rust'].value_counts()

rust
0    40596
1     6412
Name: count, dtype: int64

In [ ]:
# scp classified.csv jack@172.23.0.10:/home/jack/Documents/kona_coffee